##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# tf.data を使って pandas の DataFrame をロードする

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/pandas_dataframe"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

このチュートリアルでは、pandas のDataFrameをロードして、tf.data.Dataset にデータを読み込む例を示します。

このチュートリアルは、クリーブランドクリニック財団（the Cleveland Clinic Foundation for Heart Disease）から提供された、小さな [データセット](https://archive.ics.uci.edu/ml/datasets/heart+Disease) を使っています。このデータセット（CSV）には数百行のデータが含まれています。行は各患者を、列は様々な属性を表しています。

このデータを使って、患者が心臓病を罹患しているかどうかを判別予測することができます。なお、これは二値分類問題になります。

## pandas を使ってデータを読み込む

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import pandas as pd
import tensorflow as tf

heart データセットを含んだCSVをダウンロードします。

In [3]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


pandas を使ってCSVを読み込みます。

In [18]:
df = pd.read_csv(csv_file)

In [19]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [20]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

dataframe 内で唯一の object 型である 「thal」列を離散値に変換します。

In [21]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [22]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


## tf.data.Dataset を使ってデータをロードする

tf.data.Dataset.from_tensor_slices メソッドを使って、pandas の dataframeから値を読み込みます。

tf.data.Dataset を使う利点は、シンプルに使えて、かつ、大変効率的なデータパイプラインを構築できることです。詳しくは [loading data guide](https://www.tensorflow.org/guide/data) を参照してください。

In [23]:
target = df.pop('target')

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [25]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


pd.Series は \_\_array\_\_ プロトコルを実装しているため、np.array や tf.Tensor を使うところでは、だいたいどこでも使うことができます。

In [26]:
tf.constant(df['thal'])

<tf.Tensor: id=51, shape=(303,), dtype=int32, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3

データをシャッフルしてバッチ処理を行います。

In [27]:
train_dataset = dataset.shuffle(len(df)).batch(1)

In [67]:
train_dataset

<BatchDataset shapes: ((None, 13), (None,)), types: (tf.float64, tf.int64)>

## モデルを作成して訓練する

In [28]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [29]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/15
303/303 [==============================] - 2s 8ms/step - loss: 1.5103 - accuracy: 0.7228203 - accuracy: 0.7
Epoch 2/15
303/303 [==============================] - 0s 2ms/step - loss: 0.6130 - accuracy: 0.7195
Epoch 3/15
303/303 [==============================] - 0s 2ms/step - loss: 0.6370 - accuracy: 0.7195
Epoch 4/15
303/303 [==============================] - 0s 2ms/step - loss: 0.5848 - accuracy: 0.7162
Epoch 5/15
303/303 [==============================] - 0s 2ms/step - loss: 0.5502 - accura

## 特徴カラムの代替

Passing a dictionary as an input to a model is as easy as creating a matching dictionary of `tf.keras.layers.Input` layers, applying any pre-processing and stacking them up using the [functional api](../../guide/keras/functional.ipynb). You can use this as an alternative to [feature columns](../keras/feature_columns.ipynb).

モデルへの入力に辞書型データを渡すことは、 `tf.keras.layers.Input` に同じ型の辞書を作成し、何らかの前処理を適用して、[functional api](../../guide/keras/functional.ipynb) を使ってスタッキングすることと同様に、簡単に行うことができます。これを [特徴カラム](../keras/feature_columns.ipynb) の替わりに使うことができます。

In [30]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

tf.data を使うときに、pandas の DataFrame のカラム構造を保持する一番簡単な方法は、DataFrame を辞書型データに変換して、先頭を切り取ることです。

In [31]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [32]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

({'age': <tf.Tensor: id=14798, shape=(16,), dtype=int32, numpy=array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57])>, 'sex': <tf.Tensor: id=14806, shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])>, 'cp': <tf.Tensor: id=14801, shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3])>, 'trestbps': <tf.Tensor: id=14810, shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150])>, 'chol': <tf.Tensor: id=14800, shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168])>, 'fbs': <tf.Tensor: id=14803, shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])>, 'restecg': <tf.Tensor: id=14805, shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0])>, 'thalach': <tf.Tensor: id=14809, shape=(16,), dtype=int32, n

In [33]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 1s 31ms/step - loss: 21.6107 - accuracy: 0.7261
Epoch 2/15
19/19 [==============================] - 0s 4ms/step - loss: 9.7453 - accuracy: 0.7261
Epoch 3/15
19/19 [==============================] - 0s 4ms/step - loss: 3.0189 - accuracy: 0.4851
Epoch 4/15
19/19 [==============================] - 0s 4ms/step - loss: 2.3930 - accuracy: 0.5776
Epoch 5/15
19/19 [==============================] - 0s 4ms/step - loss: 2.2744 - accuracy: 0.5380
Epoch 6/15
19/19 [==============================] - 0s 3ms/step - loss: 2.0533 - accuracy: 0.5743
Epoch 7/15
19/19 [==============================] - 0s 3ms/step - loss: 1.8688 - accuracy: 0.5941
Epoch 8/15
19/19 [==============================] - 0s 4ms/step - loss: 1.7153 - accuracy: 0.6106
Epoch 9/15
19/19 [==============================] - 0s 4ms/step - loss: 1.5944 - accuracy: 0.6172
Epoch 10/15
19/19 [==============================] - 0s 3ms/step - loss: 1.5051 - accuracy: 0.6337
Epoch 11/15
19/19